In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Get the absolute path of the directory where the notebook is running
notebook_dir = os.getcwd()
module_path = os.path.abspath(os.path.join(notebook_dir, '..'))

if module_path not in sys.path:
    sys.path.append(module_path)

from model.helper import board_to_representation, move_to_index

# ... your other imports ...
import chess.pgn
import chess.polyglot
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
import random
import numpy as np

In [2]:
!mkdir ../data
!mkdir ../data/torch
!mkdir ../data/torch/test
!mkdir ../data/torch/train
!mkdir ../data/torch/val



mkdir: cannot create directory ‘../data’: File exists


In [3]:

!curl https://database.nikonoel.fr/lichess_elite_2025-07.zip --output ../data/lichess_elite_2025-07.zip
!unzip ../data/lichess_elite_2025-07.zip -d ../data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 79.7M  100 79.7M    0     0  68.0M      0  0:00:01  0:00:01 --:--:-- 68.0M
Archive:  ../data/lichess_elite_2025-07.zip
replace ../data/lichess_elite_2025-07.pgn? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [3]:
pgn = open("../data/lichess_elite_2025-07.pgn")

In [4]:
data_dict = {}
position_tuples = []
batch_size = 10000

In [5]:
for a in tqdm(range(75000)):
    game = chess.pgn.read_game(pgn)
    result = {"1-0":1.0, "0-1":-1.0, "1/2-1/2":0.0}[game.headers.get("Result")]
    board = game.board()
    for move in game.mainline_moves():
        _result = result#-result if board.turn == chess.BLACK else result
        _hash = chess.polyglot.zobrist_hash(board)
        data = data_dict.get(_hash)
        representation = board_to_representation(board)
        move_representation = move_to_index(move)
        if data == None:
            data_dict[_hash] = {
                "representation": representation,
                "policy": move_representation,
                "value": _result,
                "results": [_result]
            }
        else:
            data_dict[_hash]["results"].append(_result)
            data_dict[_hash]["value"] = float(np.mean(data_dict[_hash]["results"]))
 
        board.push(move)

        


  0%|          | 0/75000 [00:00<?, ?it/s]

 80%|████████  | 60063/75000 [11:30<02:51, 87.00it/s] 


KeyboardInterrupt: 

In [6]:
position_tuples = list(data_dict.values())
len(position_tuples)
    

4509269

In [7]:
temp_data, test_data = train_test_split(position_tuples, test_size=0.05, random_state=42)
train_data, val_data = train_test_split(temp_data, test_size=0.055, random_state=42)

random.shuffle(train_data)
random.shuffle(test_data)
random.shuffle(val_data)

print(len(train_data), len(test_data), len(val_data))

4048195 225464 235610


In [8]:
file_counter = 0
while len(train_data) >= batch_size:
    output_path = f"../data/torch/train/data_chunk_{file_counter}.pt"
    torch.save(train_data[:batch_size], output_path)
    train_data = train_data[batch_size:]
    file_counter += 1


file_counter = 0
while len(test_data) >= batch_size:
    output_path = f"../data/torch/test/data_chunk_{file_counter}.pt"
    torch.save(test_data[:batch_size], output_path)
    test_data = test_data[batch_size:]
    file_counter += 1


file_counter = 0
while len(val_data) >= batch_size:
    output_path = f"../data/torch/val/data_chunk_{file_counter}.pt"
    torch.save(val_data[:batch_size], output_path)
    val_data = val_data[batch_size:]
    file_counter += 1